# 뉴스 기사 GraphRAG 검색 시스템

뉴스 기사 그래프 데이터베이스에서 3가지 검색 방식을 구현하고, ToolsRetriever 기반 GraphRAG를 구축

1. **[Vector]** Content 노드 기반 벡터 검색 - 내용 유사도 기반 검색
2. **[VectorCypher]** 벡터 검색 + 카테고리 기반 관련 기사 - 복합 검색
3. **[Text2Cypher]** 자연어를 Cypher로 변환한 구조적 검색

In [1]:
import os
import neo4j
import dotenv
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.retrievers import VectorRetriever, VectorCypherRetriever, Text2CypherRetriever, ToolsRetriever
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings
from neo4j_graphrag.indexes import create_vector_index
from neo4j_graphrag.generation import RagTemplate, GraphRAG

dotenv.load_dotenv()

URI = os.getenv("NEO4J_URI", "neo4j://localhost:7687")
AUTH = ("neo4j", os.getenv("NEO4J_PASSWORD", "password"))
driver = neo4j.GraphDatabase.driver(URI, auth=AUTH)

llm = OpenAILLM(
    model_name="gpt-4o",
    model_params={"temperature": 0}
)
embedder = OpenAIEmbeddings(model="text-embedding-3-small")

## Vector Retriever

Article 노드 기반 벡터 검색 (content 노드) - 관련 기사 Article 노드 정보 추출

In [2]:
with driver.session() as session:
    result = session.run("MATCH (c:Content) WHERE c.embedding IS NULL RETURN elementId(c) AS id, c.chunk AS text")
    records = result.data()

    for record in records:
        node_id = record["id"]
        text = record["text"]
        vector = embedder.embed_query(text)

        session.run("""
        MATCH (c) WHERE elementId(c) = $id
        SET c.embedding = $embedding
        """, {"id": node_id, "embedding": vector})

In [3]:
INDEX_NAME = "content_vector_index"
DIMENSION = 1536

create_vector_index(
    driver,
    INDEX_NAME,
    label="Content",
    embedding_property="embedding",
    dimensions=DIMENSION,
    similarity_fn="cosine",
)

In [4]:
vector_retriever = VectorRetriever(
    driver=driver,
    index_name=INDEX_NAME,
    embedder=embedder
)

In [6]:
results = vector_retriever.search(query_text="북한", top_k=3)

print(f"검색 결과 수: {len(results.items)}")
for i, item in enumerate(results.items):
    print(f"\n결과 {i+1}:")
    print(f"Content: {item.content[:200]}...")
    if item.metadata:
        print(f"Metadata: {item.metadata}")

검색 결과 수: 3

결과 1:
Content: {'published_date': '2025-11-20 14:42:17', 'article_id': 'ART_018_0006168585', 'chunk': '북한 개인 사업 운영 50대 부부,\n“오만해졌다” 야외 공간서 처형\n당국 “대중 교육하기 위한 모범”\n[이데일리 권혜미 기자] 북한에서 한 부부가 사업이 성공하고 오만해졌다는 이유로 처형당한 사실...
Metadata: {'score': 0.708275556564331, 'nodeLabels': ['Content'], 'id': '4:17d251d9-0689-4c9b-8db1-e9cb07c6f7f1:1'}

결과 2:
Content: {'published_date': '2025-11-20 14:42:17', 'article_id': 'ART_018_0006168585', 'chunk': '품질이 불만스럽다. 부부의 태도가 오만하다”며 부부에게 불만을 품게 됐다.\n\n이에 당국은 8월 초 부부를 체포했고, 공동 심문 후 9월 초 사형을 선고했다. 부부의 관련인 20명도 추방 또는 재교육...
Metadata: {'score': 0.6981091499328613, 'nodeLabels': ['Content'], 'id': '4:17d251d9-0689-4c9b-8db1-e9cb07c6f7f1:2'}

결과 3:
Content: {'published_date': '2025-11-21 07:21:37', 'article_id': 'ART_052_0002276167', 'chunk': '지적이 나왔습니다.\n\n앤서니 루지에로 전 백악관 국가안보회의 북한 담당 국장은 미 싱크탱크 전략국제문제연구소 세미나에서 "한국이 핵무기를 개발하지 않겠다고 공개적으로 선언해야 한다"고 말했습니다.\...
Metadata: {'score': 0.6775124073028564, 'nodeLabels': ['Content'], 'id': '4:17d251d9-0689-4c9b-8

## VectorCypher Retriever

Article 노드 기반 벡터 검색 + 같은 카테고리의 다른 최신 뉴스 (category 노드)

In [42]:
# VectorCypher용 retrieval query 정의
# 벡터 검색으로 찾은 Content와 연결된 Article의 같은 카테고리 다른 기사들도 함께 반환
retrieval_query = """
MATCH (content:Content)<-[:HAS_CHUNK]-(article:Article)
OPTIONAL MATCH (article)-[:BELONGS_TO]->(category:Category)<-[:BELONGS_TO]-(related_article:Article)
WHERE article <> related_article

RETURN
    content.content_id AS content_id,
    content.chunk AS chunk,
    content.title AS content_title,
    article.article_id AS article_id,
    article.title AS article_title,
    article.url AS article_url,
    article.published_date AS article_date,
    category.name AS category_name,
    collect(DISTINCT {
        article_id: related_article.article_id,
        title: related_article.title,
        url: related_article.url,
        published_date: related_article.published_date
    }) AS related_articles

ORDER BY article.published_date DESC
LIMIT 1
"""

In [43]:
vector_cypher_retriever = VectorCypherRetriever(
    driver=driver,
    index_name=INDEX_NAME,
    retrieval_query=retrieval_query,
    embedder=embedder
)

In [16]:
results = vector_cypher_retriever.search(query_text="삼성전자")

print(f"검색 결과 수: {len(results.items)}")
for i, item in enumerate(results.items):
    print(f"\n결과 {i+1}:")
    print(f"Content: {item.content[:200]}...")
    print(item.content)

검색 결과 수: 1

결과 1:
Content: <Record content_id='ART_138_0002210299_chunk_0' chunk="삼성전자, 정기 사장단 인사…노태문, 직무대행 떼고 '대표이사 사장'\n노태문 삼성전자 사장이 대표이사 겸 DX부문장으로 승진했다. 동시에 MX 사업부장 직책은 유지한다. 사진은 올해 9월 독일 베를린에서 열린 IFA 2025 삼성전자 기자 간담회에서 노태문 ...
<Record content_id='ART_138_0002210299_chunk_0' chunk="삼성전자, 정기 사장단 인사…노태문, 직무대행 떼고 '대표이사 사장'\n노태문 삼성전자 사장이 대표이사 겸 DX부문장으로 승진했다. 동시에 MX 사업부장 직책은 유지한다. 사진은 올해 9월 독일 베를린에서 열린 IFA 2025 삼성전자 기자 간담회에서 노태문 사장이 발표하는 모습이다. [사진=옥송이기자]\n\n\n[디지털데일리 옥송이 기자] “AI 드리븐 컴퍼니(AI Driven Company)로 거듭나고자 한다. 비즈니스의 근본부터 AI로 혁신하겠다. 이를 통해 삼성은 AI를 가장 잘 활용하는 회사, AI로 일하고 성장하는 회사로 전환하겠다.”\n\n노태문 삼성전자 사장이 대표이사로 선임됐다. 자연스럽게 올해 3월부터 8개월간 유지했던 디바이스경험(DX)부문장 ‘직무대행’을 졸업하고 정식 DX부문장에 올랐다. 이로써 삼성전자는 반도체(DS) 사업 전영현 부회장, 모바일·가전(DX) 사업 노태문 사장으로 구성된 DS·DX 대표이사 투톱 체제를 공고히 하며 경영 안정에 주력한다.\n\n삼성전자는 2" content_title='[삼성리더십] "근본부터 AI 기업" 강조한 노태문, DX·MX 총괄…\'따로 또 같이\' AI 전략 가속' article_id='ART_138_0002210299' article_title='[삼성리더십] "근본부터 AI 기업" 강조한 노태문, DX·MX 총괄…\'따로 또 같이\' AI 전략 가속' article_url='https:/

In [17]:
import re
import json
import ast

def extract_field(text, name):
    pattern = rf"{name}\s*=\s*(?P<val>'[^']*'|\"[^\"]*\"|'''(?:.|\n)*?'''|\"\"\"(?:.|\n)*?\"\"\")"
    m = re.search(pattern, text)

    val = m.group("val")

    return val[1:-1]

def extract_related(text):
    m = re.search(r"related_articles\s*=\s*(\[.*?\])", text, re.DOTALL)
    raw = m.group(1)

    return ast.literal_eval(raw)

def parse_content(text):
    fields = [
        'content_id', 'chunk', 'content_title',
        'article_id', 'article_title', 'article_url',
        'article_date', 'category_name'
    ]

    extracted = {f: extract_field(text, f) for f in fields}
    extracted['related_articles'] = extract_related(text)

    if extracted['chunk']:
        extracted['chunk'] = extracted['chunk'].replace('\\n', '\n')

    return extracted

result = parse_content(str(item.content))

print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "content_id": "ART_138_0002210299_chunk_0",
  "chunk": "삼성전자, 정기 사장단 인사…노태문, 직무대행 떼고 '대표이사 사장'\n노태문 삼성전자 사장이 대표이사 겸 DX부문장으로 승진했다. 동시에 MX 사업부장 직책은 유지한다. 사진은 올해 9월 독일 베를린에서 열린 IFA 2025 삼성전자 기자 간담회에서 노태문 사장이 발표하는 모습이다. [사진=옥송이기자]\n\n\n[디지털데일리 옥송이 기자] “AI 드리븐 컴퍼니(AI Driven Company)로 거듭나고자 한다. 비즈니스의 근본부터 AI로 혁신하겠다. 이를 통해 삼성은 AI를 가장 잘 활용하는 회사, AI로 일하고 성장하는 회사로 전환하겠다.”\n\n노태문 삼성전자 사장이 대표이사로 선임됐다. 자연스럽게 올해 3월부터 8개월간 유지했던 디바이스경험(DX)부문장 ‘직무대행’을 졸업하고 정식 DX부문장에 올랐다. 이로써 삼성전자는 반도체(DS) 사업 전영현 부회장, 모바일·가전(DX) 사업 노태문 사장으로 구성된 DS·DX 대표이사 투톱 체제를 공고히 하며 경영 안정에 주력한다.\n\n삼성전자는 2",
  "content_title": "[삼성리더십] \"근본부터 AI 기업\" 강조한 노태문, DX·MX 총괄…\\",
  "article_id": "ART_138_0002210299",
  "article_title": "[삼성리더십] \"근본부터 AI 기업\" 강조한 노태문, DX·MX 총괄…\\",
  "article_url": "https://n.news.naver.com/mnews/article/138/0002210299",
  "article_date": "2025-11-21 13:15:09",
  "category_name": "IT/과학",
  "related_articles": [
    {
      "published_date": "2025-11-21 10:50:13",
      "article_id": "ART_009_0005593962

## Text2Cypher Retriever

In [18]:
def get_schema():
    """Neo4j 데이터베이스의 스키마 정보를 가져옵니다"""
    with driver.session() as session:
        # 노드 라벨과 속성 정보
        node_info = session.run("""
            CALL db.schema.nodeTypeProperties()
            YIELD nodeType, propertyName, propertyTypes
            RETURN nodeType, collect(propertyName) as properties
        """).data()

        # 관계 정보
        rel_info = session.run("""
            CALL db.schema.relTypeProperties()
            YIELD relType, propertyName, propertyTypes
            RETURN relType, collect(propertyName) as properties
        """).data()

        # 관계 패턴 정보
        patterns = session.run("""
            MATCH (n)-[r]->(m)
            RETURN DISTINCT labels(n)[0] as source, type(r) as relationship, labels(m)[0] as target
            LIMIT 20
        """).data()

        schema_text = "=== Neo4j Schema ===\n"

        # 노드 정보 추가
        schema_text += "\n노드 타입:\n"
        for node in node_info:
            schema_text += f"- {node['nodeType']}: {node['properties']}\n"

        # 관계 정보 추가
        schema_text += "\n관계 패턴:\n"
        for pattern in patterns:
            schema_text += f"- ({pattern['source']})-[:{pattern['relationship']}]->({pattern['target']})\n"

        return schema_text

neo4j_schema = get_schema()
print(neo4j_schema)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: '\n            CALL db.schema.relTypeProperties()\n            YIELD relType, propertyName, propertyTypes\n            RETURN relType, collect(propertyName) as properties\n        '


=== Neo4j Schema ===

노드 타입:
- :`Content`: ['article_id', 'content_id', 'title', 'url', 'published_date', 'chunk', 'chunk_index', 'embedding']
- :`Media`: ['name']
- :`Category`: ['name']
- :`Article`: ['article_id', 'title', 'url', 'published_date']

관계 패턴:
- (Article)-[:HAS_CHUNK]->(Content)
- (Article)-[:BELONGS_TO]->(Category)
- (Media)-[:PUBLISHED]->(Article)



In [19]:
examples = [
    """
    USER INPUT: 경제 분야의 최신 뉴스 알려주세요
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category {name: "경제"})
    RETURN a.article_id, a.title, a.url, a.published_date
    ORDER BY a.published_date DESC
    LIMIT 10
    """,

    """
    USER INPUT: 매일경제에서 나온 최신 뉴스 3개 보여주세요
    CYPHER QUERY:
    MATCH (m:Media {name: "매일경제"})-[:PUBLISHED]->(a:Article)
    RETURN a.article_id, a.title, a.url, a.published_date
    ORDER BY a.published_date DESC
    LIMIT 3
    """,

    """
    USER INPUT: 2025년 11월 1일 이후에 발행된 정치 관련 기사는 몇 개나 되나요?
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category {name: "정치"})
    WHERE a.published_date >= "2025-11-01"
    RETURN count(a) as article_count
    """,

    """
    USER INPUT: 카테고리별 기사 개수를 알려주세요
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category)
    RETURN c.name as category, count(a) as article_count
    ORDER BY article_count DESC
    """,

    """
    USER INPUT: 11월 2일에 발행된 기사 중 정치 분야는?
    CYPHER QUERY:
    MATCH (a:Article)-[:BELONGS_TO]->(c:Category {name: "정치"})
    WHERE a.published_date STARTS WITH "2025-11-02"
    RETURN a.article_id, a.title, a.url, a.published_date
    ORDER BY a.published_date DESC
    """,
]

text2cypher_retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,
    neo4j_schema=neo4j_schema,
    examples=examples,
)

In [20]:
results = text2cypher_retriever.search(query_text="정치 카테고리의 최신 기사 5개를 보여주세요")
for j, item in enumerate(results.items):
    print(f"  결과 {j+1}:")
    print(f"  Content: {item.content}")
    if item.metadata:
        print(f"  Metadata: {item.metadata}")

  결과 1:
  Content: <Record a.article_id='ART_015_0005214392' a.title='李대통령 "민주주의의 우뚝 선 봉우리"…김영삼 전 대통령 추모' a.url='https://n.news.naver.com/mnews/article/015/0005214392' a.published_date='2025-11-21 11:07:11'>
  결과 2:
  Content: <Record a.article_id='ART_449_0000327216' a.title='이 대통령 지지율 60%…민주 43%·국힘 24%' a.url='https://n.news.naver.com/mnews/article/449/0000327216' a.published_date='2025-11-21 10:52:16'>
  결과 3:
  Content: <Record a.article_id='ART_016_0002561105' a.title='‘공천 혁명’이냐 ‘연임 노림수’냐…정청래 ‘1인1표제’ 실현 임박[이런정치]' a.url='https://n.news.naver.com/mnews/article/016/0002561105' a.published_date='2025-11-21 10:31:11'>
  결과 4:
  Content: <Record a.article_id='ART_277_0005682723' a.title='송언석 "정부혁신 TF, 야만적 공무원 줄세우기...김문기법 추진"' a.url='https://n.news.naver.com/mnews/article/277/0005682723' a.published_date='2025-11-21 10:18:33'>
  결과 5:
  Content: <Record a.article_id='ART_421_0008616962' a.title='李대통령, 남아공서 프랑스·독일과 정상회담…"국제정세 변화 대응"' a.url='https://n.news.naver.com/mnews/article/421/00086

In [21]:
results = text2cypher_retriever.search(query_text="언론사별로 기사 개수를 알려주세요")
for j, item in enumerate(results.items):
    print(f"  결과 {j+1}:")
    print(f"  Content: {item.content}")
    if item.metadata:
        print(f"  Metadata: {item.metadata}")

  결과 1:
  Content: <Record media='연합뉴스' article_count=5>
  결과 2:
  Content: <Record media='이데일리' article_count=4>
  결과 3:
  Content: <Record media='뉴스1' article_count=4>
  결과 4:
  Content: <Record media='조선비즈' article_count=4>
  결과 5:
  Content: <Record media='한국경제' article_count=4>
  결과 6:
  Content: <Record media='헤럴드경제' article_count=4>
  결과 7:
  Content: <Record media='YTN' article_count=3>
  결과 8:
  Content: <Record media='아시아경제' article_count=3>
  결과 9:
  Content: <Record media='뉴시스' article_count=3>
  결과 10:
  Content: <Record media='경향신문' article_count=2>
  결과 11:
  Content: <Record media='매일경제' article_count=2>
  결과 12:
  Content: <Record media='한국경제TV' article_count=2>
  결과 13:
  Content: <Record media='서울신문' article_count=2>
  결과 14:
  Content: <Record media='코메디닷컴' article_count=2>
  결과 15:
  Content: <Record media='디지털타임스' article_count=2>
  결과 16:
  Content: <Record media='채널A' article_count=1>
  결과 17:
  Content: <Record media='노컷뉴스' article_count=1>
  결과 18:
  Content: 

## Tools Retriever (Vector + VectorCypher + Text2Cypher)

[Neo4j GraphRAG 파이썬 패키지 위키독스 ToolsRetriever](https://wikidocs.net/307512)

In [22]:
vector_tool = vector_retriever.convert_to_tool(
    name="vector_retriever",
    description="벡터 기반 검색으로 뉴스기사에 등장하는 내용 텍스트를 기반으로 검색할 때 사용합니다."
)
vector_cypher_tool = vector_cypher_retriever.convert_to_tool(
    name="vectorcypher_retriever",
    description="벡터 검색으로 찾아진 Content와 연결된 Article을 기준으로, 그 기사의 상세한 정보는 물론 같은 카테고리의 다른 기사들을 함께 반환합니다."
)
text2cypher_tool = text2cypher_retriever.convert_to_tool(
    name="text2cypher_retriever",
    description="text2cypher 검색 기반으로 언론사, 분야별 기사 등 엔티티 혹은 속성을 기반으로 정보를 찾을 때 사용합니다."
)

In [23]:
tools_retriever = ToolsRetriever(
    driver=driver,
    llm=llm,
    tools=[vector_tool, vector_cypher_tool, text2cypher_tool],
)

In [25]:
results = tools_retriever.search("정치 카테고리의 최신 뉴스 알려줘")
print(results.items[0].content)
print(results.items[0].metadata)

<Record content='<Record a.article_id=\'ART_015_0005214392\' a.title=\'李대통령 "민주주의의 우뚝 선 봉우리"…김영삼 전 대통령 추모\' a.url=\'https://n.news.naver.com/mnews/article/015/0005214392\' a.published_date=\'2025-11-21 11:07:11\'>' tool_name='text2cypher_retriever' metadata={'tool': 'text2cypher_retriever'}>
{'tool': 'text2cypher_retriever'}


In [26]:
prompt_template = RagTemplate(
    template="""당신은 뉴스 기사 정보를 제공하는 전문 어시스턴트입니다.

질문: {query_text}

검색된 문서 정보:
{context}

지침:
1. 제공된 검색 결과(Context)의 내용을 충실히 사용하여 답변하세요.
2. 답변에는 반드시 관련 뉴스 기사의 **제목(title)**과 **URL(url)**을 포함해야 합니다.
3. 여러 기사가 검색된 경우, 각 기사의 출처를 명확히 구분하여 제시하세요.
4. 검색 결과에 없는 내용은 추측하지 마세요.
5. 가능하면 발행일(published_date)과 카테고리(category_name)도 함께 언급하세요.

답변 형식 예시:
- [기사 제목] (카테고리명, 발행일)
  URL: [기사 URL]
  내용: [관련 내용 요약]

답변:""",
    expected_inputs=["context", "query_text"]
)

In [27]:
graphrag = GraphRAG(
    llm=llm,
    retriever=tools_retriever,
    prompt_template=prompt_template
)

In [28]:
result = graphrag.search(query_text="정치 카테고리의 최신 기사", return_context=True)

print("============== [ 답변 결과 ] ==============")
print(result.answer)
print("==========================================")

if hasattr(result, 'retriever_result') and result.retriever_result:
    print(f"\n검색 결과 ({len(result.retriever_result.items)}개):")
    for idx, item in enumerate(result.retriever_result.items, 1):
        print(f"\n--- 결과 {idx} ---")
        print(f"Content: {item.content}")

        if item.metadata:
            print(f"Metadata: {item.metadata}")

============== [ 답변 결과 ] ==============
- [李대통령 "민주주의의 우뚝 선 봉우리"…김영삼 전 대통령 추모] (정치, 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/015/0005214392](https://n.news.naver.com/mnews/article/015/0005214392)
  내용: 이 대통령이 김영삼 전 대통령을 추모하며 민주주의의 중요성을 강조한 기사입니다.

- [이 대통령 지지율 60%…민주 43%·국힘 24%] (정치, 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/449/0000327216](https://n.news.naver.com/mnews/article/449/0000327216)
  내용: 이 대통령의 지지율이 60%를 기록하며, 민주당과 국민의힘의 지지율을 비교한 기사입니다.

- [‘공천 혁명’이냐 ‘연임 노림수’냐…정청래 ‘1인1표제’ 실현 임박[이런정치]] (정치, 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/016/0002561105](https://n.news.naver.com/mnews/article/016/0002561105)
  내용: 정청래 의원의 '1인1표제' 실현이 임박하며, 이를 둘러싼 정치적 논란을 다룬 기사입니다.

- [송언석 "정부혁신 TF, 야만적 공무원 줄세우기...김문기법 추진"] (정치, 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/277/0005682723](https://n.news.naver.com/mnews/article/277/0005682723)
  내용: 송언석 의원이 정부혁신 TF의 문제점을 지적하며 김문기법 추진을 언급한 기사입니다.

- [李대통령, 남아공서 프랑스·독일과 정상회담…"국제정세 변화 대응

In [29]:
result = graphrag.search(query_text="트럼프 대통령 관련 내용", return_context=True)

print("============== [ 답변 결과 ] ==============")
print(result.answer)
print("==========================================")

if hasattr(result, 'retriever_result') and result.retriever_result:
    print(f"\n검색 결과 ({len(result.retriever_result.items)}개):")
    for idx, item in enumerate(result.retriever_result.items, 1):
        print(f"\n--- 결과 {idx} ---")
        print(f"Content: {item.content}")

        if item.metadata:
            print(f"Metadata: {item.metadata}")

============== [ 답변 결과 ] ==============
- [중·러, 고위급 외교·군사 회담 열고 미사일 방어 논의] (발행일: 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/032/0003410297](https://n.news.naver.com/mnews/article/032/0003410297)
  내용: 도널드 트럼프 미국 대통령이 핵무기 실험을 재개하겠다고 밝히고 차세대 미사일 방어망 ‘골든돔’ 구축 계획을 추진하는 상황에서, 중국과 러시아가 고위급 외교·군사 회담을 열고 미사일 방어를 논의했습니다. 트럼프 대통령은 중국, 러시아와 함께 비핵화 계획을 추진할 수 있다고도 밝혔습니다.

- [트럼프, ‘고물가 압박’ 속 브라질 농산물에 40% 추가관세 면제] (발행일: 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/016/0002561078](https://n.news.naver.com/mnews/article/016/0002561078)
  내용: 도널드 트럼프 미국 대통령이 고물가로 인한 정치적 압박을 받는 가운데, 브라질 주요 농산물에 대해 부과했던 40%의 추가 관세를 면제했습니다. 이로 인해 브라질산 농산물에 대한 미국의 관세율은 0%가 되었습니다.

검색 결과 (5개):

--- 결과 1 ---
Content: <Record content='{\'published_date\': \'2025-11-21 03:06:56\', \'article_id\': \'ART_052_0002276132\', \'chunk\': "이집트를 공식 방문 중인 이재명 대통령은 남북 적대와 대결의 시대를 끝내고, 평화 공존과 공동 성장의 새 시대를 열어가고자 한다고 강조했습니다.\\n\\n이 대통령은 현지 시각 20일, 이집트 카이로 대학을 방문한 자리에서 우선 가능한 분야에서 남북 간 교류·협력을 단계적으로 확대하겠다며 이같이 연설했

In [49]:
result = graphrag.search(query_text="삼성전자 승진 관련 내용 기사 전체 정보", return_context=True)

print("============== [ 답변 결과 ] ==============")
print(result.answer)
print("==========================================")

if hasattr(result, 'retriever_result') and result.retriever_result:
    print(f"\n검색 결과 ({len(result.retriever_result.items)}개):")
    for idx, item in enumerate(result.retriever_result.items, 1):
        print(f"\n--- 결과 {idx} ---")
        print(f"Content: {item.content}")

        if item.metadata:
            print(f"Metadata: {item.metadata}")

============== [ 답변 결과 ] ==============
- [삼성리더십] "근본부터 AI 기업" 강조한 노태문, DX·MX 총괄…'따로 또 같이' AI 전략 가속 (IT/과학, 2025-11-21)
  URL: [https://n.news.naver.com/mnews/article/138/0002210299](https://n.news.naver.com/mnews/article/138/0002210299)
  내용: 노태문 삼성전자 사장이 대표이사로 선임되었으며, 디바이스경험(DX)부문장 직무대행을 졸업하고 정식 DX부문장에 올랐습니다. 삼성전자는 반도체(DS) 사업 전영현 부회장과 모바일·가전(DX) 사업 노태문 사장으로 구성된 DS·DX 대표이사 투톱 체제를 공고히 하며 경영 안정에 주력하고 있습니다. 노태문 사장은 AI 드리븐 컴퍼니로의 전환을 강조하며, 비즈니스의 근본부터 AI로 혁신하겠다는 계획을 밝혔습니다.

검색 결과 (1개):

--- 결과 1 ---
Content: <Record content='<Record content_id=\'ART_138_0002210299_chunk_0\' chunk="삼성전자, 정기 사장단 인사…노태문, 직무대행 떼고 \'대표이사 사장\'\\n노태문 삼성전자 사장이 대표이사 겸 DX부문장으로 승진했다. 동시에 MX 사업부장 직책은 유지한다. 사진은 올해 9월 독일 베를린에서 열린 IFA 2025 삼성전자 기자 간담회에서 노태문 사장이 발표하는 모습이다. [사진=옥송이기자]\\n\\n\\n[디지털데일리 옥송이 기자] “AI 드리븐 컴퍼니(AI Driven Company)로 거듭나고자 한다. 비즈니스의 근본부터 AI로 혁신하겠다. 이를 통해 삼성은 AI를 가장 잘 활용하는 회사, AI로 일하고 성장하는 회사로 전환하겠다.”\\n\\n노태문 삼성전자 사장이 대표이사로 선임됐다. 자연스럽게 올해 3월부터 8개월간 유지했던 디바이스경험(DX)부문장 ‘직무대행’을 졸업하고 정식 DX부문장에 올랐다. 이로써 삼성